In [1]:
from transformers import Swinv2Config, Swinv2Model, AutoFeatureExtractor
import transformers
import os, json, glob, requests, torch, cv2
import numpy as np
from PIL import Image
from torch.utils.data import DataLoader, Dataset
import pandas as pd
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
from sklearn.metrics import accuracy_score
from tensorflow.keras.preprocessing import image
from transformers import Swinv2ForImageClassification

c:\Users\User\anaconda3\envs\my_env_py311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#set default path
TRAIN_JSON_PATH = 'data/label/train/'
VALID_JSON_PATH = 'data/label/val/'

TRAIN_IMG_PATH = 'data/train/'
VALID_IMG_PATH = 'data/val/'

TRAIN_SEGMENT_PATH = 'data/segmentation/train/'
VALID_SEGMENT_PATH = 'data/segmentation/val/'

IMG_ROWS = 256
IMG_COLS = 256

In [3]:
#To get json list that contains info about the pictures.
def get_json_list_1d(json_folder_path): #returns a 1 dim array
    temp_loader = []
    json_train = []
    emotion_list_json = os.listdir(json_folder_path)

    for emotion in emotion_list_json:
        with open (json_folder_path + emotion, "r", encoding='euc-kr') as f:
            temp_loader.append(json.load(f))     

    for i in range(len(temp_loader)):
        json_train += temp_loader[i]     

    return json_train


def get_json_list_2d(json_folder_path): #returns 2 dim array
    json_train = []
    emotion_list_json = os.listdir(json_folder_path)

    for emotion in emotion_list_json:
        with open (json_folder_path + emotion, "r", encoding='euc-kr') as f:
            json_train.append(json.load(f))     

    return json_train





In [4]:
#call for the functions
json_train = get_json_list_1d(TRAIN_JSON_PATH)
json_val = get_json_list_1d(VALID_JSON_PATH)
print(json_train[0].keys()) #see what keys are available
print(len(json_train))
print(len(json_val))

#test
# json_train[0]['annot_A']['boxes']

# for coord in json_train[0]['annot_A']['boxes']:
#     print(json_train[0]['annot_A']['boxes'][coord])

dict_keys(['filename', 'gender', 'age', 'isProf', 'faceExp_uploader', 'bg_uploader', 'annot_A', 'annot_B', 'annot_C'])
5382
1049


In [5]:
json_train_2d = get_json_list_2d(TRAIN_JSON_PATH)
json_val_2d = get_json_list_2d(VALID_JSON_PATH)
#0 = 분노, 1 = 기쁨, 2 = 당황, 3 = 슬픔
#0 = anger, 1 = happy, 2 = panic, 3 = sadness

print(os.listdir(TRAIN_IMG_PATH))

['anger', 'happy', 'panic', 'sadness']


In [6]:
print(len(json_val_2d[1]))
# len(json_train)
print(len(os.listdir(VALID_IMG_PATH + 'happy/')))

303
303


파일 JSON 이랑 비교해서 빠진거 있나 확인

In [13]:
filenames_json = [entry['filename'] for entry in json_val_2d[1]]
filenames_folder = os.listdir(VALID_IMG_PATH + 'happy/')


print(list( set(filenames_json) - set(filenames_folder)))
print(len(list( set(filenames_json) - set(filenames_folder))))

[]
0


In [14]:
#checking if that file exist in the path
temp = r'C:\Users\User\Downloads\Project3\val\happy'
filenames_folder = os.listdir(temp)

'7uji1c8ab7778bc286c3cd90077847f9a906f00b210838ba57877a29abd60ffix.jpg' in filenames_folder

False

get all images and put them into a list

In [116]:
#0 = 분노, 1 = 기쁨, 2 = 당황, 3 = 슬픔

# feature_extractor = AutoFeatureExtractor.from_pretrained("microsoft/swinv2-tiny-patch4-window8-256")

#designed for 2D
def load_image_and_label_2D(json_list, img_path):
    emotion_list = os.listdir(img_path)
    img_file = []
    label = []
    
    eng_to_num = {'anger'   : '0',
                 'happy'   : '1',
                 'panic'   : '2',
                 'sadness' : '3'             
                 }
    i = 0
    for emotion in emotion_list:
        for entry in range(len(json_list[i])):
            try:
                img_dir = img_path + emotion + '/' + json_list[i][entry]['filename']
                img = np.array(image.load_img(img_dir, target_size=(IMG_ROWS, IMG_COLS)))/255
                img_file.append(img)
                label.append(int(eng_to_num[emotion]))
            except Exception:
                print("Emotion " + emotion + " " + json_list[i][entry]['filename'] + " Not available")
                pass
        i += 1
    return img_file, label



In [7]:
#0 = 분노, 1 = 기쁨, 2 = 당황, 3 = 슬픔

# feature_extractor = AutoFeatureExtractor.from_pretrained("microsoft/swinv2-tiny-patch4-window8-256")

#designed for normal json file
def load_image_and_label_1D(json_list, img_path):
    img_file = []
    label_list = []
    
    eng_to_num = { '분노'   : '0',
                   '기쁨'   : '1',
                   '당황'   : '2',
                   '슬픔'   : '3'             
                 }
    kr_to_en = { '분노'    : 'anger',
                 '기쁨'   : 'happy',
                 '당황'   : 'panic',
                 '슬픔'   : 'sadness'             
                 }

    
    for entry in range(len(json_list)):
        try:
            label = json_list[entry]['faceExp_uploader']
            img_dir = img_path + kr_to_en[label] + '/' + json_list[entry]['filename']
            # img = np.array(image.load_img(img_dir, target_size=(IMG_ROWS, IMG_COLS)))/255
            img = image.load_img(img_dir, target_size=(IMG_ROWS, IMG_COLS))
            img_file.append(img)
            label_list.append(int(eng_to_num[label]))
        except Exception:
            print("Emotion " + label + " " + json_list[entry]['filename'] + " Not available")
            pass

    return img_file, label_list

In [8]:
train_img, train_label = load_image_and_label_1D(json_train, TRAIN_IMG_PATH)

Emotion 당황 twdva8d723865fa1005873a6967797c75f3dd83197f13f7daa3e3926947071al6.jpg Not available
Emotion 당황 5yid6d7ae32bf14b1343da46cfdb3fdffc8e1c64baf4fb1b175690f648a870v69.jpg Not available
Emotion 당황 bpv531e5cebdb1c00085158bb0677c5596ccabf1c8a5cb0007991571bf306wcd4.jpg Not available
Emotion 당황 s1q397a38900c1e1f950496109900e7c01f0630459790d8590010655c4090vlh0.jpg Not available
Emotion 당황 7m2k46c05803855399ea1ee1911875ee59e05640c93135c0bc91b5660456cdo8k.jpg Not available
Emotion 당황 n55yc741f09701506c7eb5ba30533db35c905ab5bf0597b5652ece03c5950ozv1.jpg Not available
Emotion 당황 bid7b3c93409e1d83514689337f6d3d63a338339d71c9ce7fded153d3576fk2ml.jpg Not available
Emotion 당황 if0lfc4b517c6597e3656dbea350e406e1b5ae4e175e460597064875909d9t40p.jpg Not available
Emotion 당황 58w153f3f5ade2ad20ec19e6744ff2ff0a54ba3520ffbef36f3f3a47af75fxuf5.jpg Not available
Emotion 당황 lhyp01dff4faf13d1b4319722dc31bb3a104a3878ab3816a03bb8acdfa3d3eomk.jpg Not available
Emotion 당황 vba0d31d23b44711180462388c8aba4424a0aa2

In [117]:
# train_img, train_label = load_image_and_label_2D(json_train_2d, TRAIN_IMG_PATH)


Emotion panic twdva8d723865fa1005873a6967797c75f3dd83197f13f7daa3e3926947071al6.jpg Not available
Emotion panic 5yid6d7ae32bf14b1343da46cfdb3fdffc8e1c64baf4fb1b175690f648a870v69.jpg Not available
Emotion panic bpv531e5cebdb1c00085158bb0677c5596ccabf1c8a5cb0007991571bf306wcd4.jpg Not available
Emotion panic s1q397a38900c1e1f950496109900e7c01f0630459790d8590010655c4090vlh0.jpg Not available
Emotion panic 7m2k46c05803855399ea1ee1911875ee59e05640c93135c0bc91b5660456cdo8k.jpg Not available
Emotion panic n55yc741f09701506c7eb5ba30533db35c905ab5bf0597b5652ece03c5950ozv1.jpg Not available
Emotion panic bid7b3c93409e1d83514689337f6d3d63a338339d71c9ce7fded153d3576fk2ml.jpg Not available
Emotion panic if0lfc4b517c6597e3656dbea350e406e1b5ae4e175e460597064875909d9t40p.jpg Not available
Emotion panic 58w153f3f5ade2ad20ec19e6744ff2ff0a54ba3520ffbef36f3f3a47af75fxuf5.jpg Not available
Emotion panic lhyp01dff4faf13d1b4319722dc31bb3a104a3878ab3816a03bb8acdfa3d3eomk.jpg Not available
Emotion panic vba0d3

In [9]:
val_img, val_label = load_image_and_label_1D(json_val, VALID_IMG_PATH)


In [112]:
print(len(train_img))
print(type(train_label[0]))
print(len(val_img))
print(train_img[0].shape)

5364
<class 'str'>
1049
(256, 256, 3)


Modeling

In [10]:
from transformers import AutoImageProcessor, Swinv2ForImageClassification

In [11]:
model_name = "microsoft/swinv2-tiny-patch4-window8-256"
# image_processor = AutoImageProcessor.from_pretrained(model_name)
# # model = Swinv2ForImageClassification.from_pretrained(model_name, ignore_mismatched_sizes=True, num_labels=4)
# model = Swinv2ForImageClassification.from_pretrained(model_name)

In [12]:
# inputs = image_processor(train_img, return_tensors="pt", do_rescale=False)

In [101]:
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# model = model.to(device)


In [200]:
# train_dataset = (train_img, train_label)
# val_dataset = (val_img, val_label)
# train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
# val_loader = DataLoader(val_dataset, batch_size=8, shuffle=True)

In [127]:
# def train_model(model, train_loader, val_loader, epochs=10, lr=1e-5):
#     optimizer = optim.RMSprop(model.parameters(), lr=lr)
#     criterion = nn.CrossEntropyLoss()

#     for epoch in range(epochs):
#         model.train()
#         train_loss = 0
#         for images, labels in train_loader:
#             images, labels = images.to(device), labels.to(device)

#             optimizer.zero_grad()
#             outputs = model(images).logits
#             loss = criterion(outputs, labels)
#             loss.backward()
#             optimizer.step()
#             train_loss += loss.item()

#         model.eval()
#         val_loss = 0
#         val_preds, val_targets = [], []
#         with torch.no_grad():
#             for images, labels in val_loader:
#                 images, labels = images.to(device), labels.to(device)
#                 outputs = model(images).logits
#                 loss = criterion(outputs, labels)
#                 val_loss += loss.item()

#                 preds = outputs.argmax(dim=1)
#                 val_preds.extend(preds.cpu().numpy())
#                 val_targets.extend(labels.cpu().numpy())

#         train_loss /= len(train_loader)
#         val_loss /= len(val_loader)
#         val_accuracy = accuracy_score(val_targets, val_preds)

#         print(f'Epoch {epoch+1}/{epochs}')
#         print(f'Train Loss: {train_loss:.4f}')
#         print(f'Val Loss: {val_loss:.4f}')
#         print(f'Val Accuracy: {val_accuracy:.4f}')

#     return model

# trained_model = train_model(model, train_loader, val_loader, epochs=10, lr=1e-5)

TypeError: only length-1 arrays can be converted to Python scalars

In [13]:
train_df = pd.DataFrame(columns=['image', 'label'])
train_df['image'] = train_img
train_df['label'] = train_label

val_df = pd.DataFrame(columns=['image', 'label'])
val_df['image'] = val_img
val_df['label'] = val_label
# df

In [14]:
train_dict = train_df.to_dict('records')
val_dict = val_df.to_dict('records')

# train_dict[0]['image']
# train_dict
# print(type(train_dict[0]))

In [15]:
label2id = {       '분노'   : '0',
                   '기쁨'   : '1',
                   '당황'   : '2',
                   '슬픔'   : '3'             
                 }
id2label = {
    '0' : '분노',
    '1' : '기쁨',
    '2' : '당황',
    '3' : '슬픔'
}
id2label

{'0': '분노', '1': '기쁨', '2': '당황', '3': '슬픔'}

In [16]:
from transformers import AutoImageProcessor

image_processor  = AutoImageProcessor.from_pretrained(model_name)
image_processor

c:\Users\User\anaconda3\envs\my_env_py311\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration. Please open a PR/issue to update `preprocessor_config.json` to use `image_processor_type` instead of `feature_extractor_type`. This warning will be removed in v4.40.


ViTImageProcessor {
  "_valid_processor_keys": [
    "images",
    "do_resize",
    "size",
    "resample",
    "do_rescale",
    "rescale_factor",
    "do_normalize",
    "image_mean",
    "image_std",
    "return_tensors",
    "data_format",
    "input_data_format"
  ],
  "do_normalize": true,
  "do_rescale": true,
  "do_resize": true,
  "image_mean": [
    0.485,
    0.456,
    0.406
  ],
  "image_processor_type": "ViTImageProcessor",
  "image_std": [
    0.229,
    0.224,
    0.225
  ],
  "resample": 3,
  "rescale_factor": 0.00392156862745098,
  "size": {
    "height": 256,
    "width": 256
  }
}

In [17]:
from torchvision.transforms import (
    CenterCrop,
    Compose,
    Normalize,
    RandomHorizontalFlip,
    RandomResizedCrop,
    Resize,
    ToTensor,
)

normalize = Normalize(mean=image_processor.image_mean, std=image_processor.image_std)
if "height" in image_processor.size:
    size = (image_processor.size["height"], image_processor.size["width"])
    crop_size = size
    max_size = None
elif "shortest_edge" in image_processor.size:
    size = image_processor.size["shortest_edge"]
    crop_size = (size, size)
    max_size = image_processor.size.get("longest_edge")

train_transforms = Compose(
        [
            RandomResizedCrop(crop_size),
            RandomHorizontalFlip(),
            ToTensor(),
            normalize,
        ]
    )

val_transforms = Compose(
        [
            Resize(size),
            CenterCrop(crop_size),
            ToTensor(),
            normalize,
        ]
    )

def preprocess_train(example_batch):
    """Apply train_transforms across a batch."""
    example_batch["pixel_values"] = [
        train_transforms(image.convert("RGB")) for image in example_batch["image"]
    ]
    return example_batch

def preprocess_val(example_batch):
    """Apply val_transforms across a batch."""
    example_batch["pixel_values"] = [val_transforms(image.convert("RGB")) for image in example_batch["image"]]
    return example_batch

In [18]:
from datasets import Dataset
# train_ds = Dataset.from_pandas(pd.DataFrame(data=train_dict))
train_ds = Dataset.from_list(train_dict)
val_ds = Dataset.from_list(val_dict)
train_ds

Dataset({
    features: ['image', 'label'],
    num_rows: 5364
})

In [19]:
train_ds.set_transform(preprocess_train)
val_ds.set_transform(preprocess_val)

In [20]:
train_ds[0]

{'image': <PIL.PngImagePlugin.PngImageFile image mode=RGB size=256x256>,
 'label': 0,
 'pixel_values': tensor([[[-1.1075, -1.1418, -1.1247,  ..., -1.0733, -1.0390, -1.0904],
          [-1.1247, -1.1247, -1.1075,  ..., -1.0904, -1.0562, -1.0733],
          [-1.1075, -1.1075, -1.1075,  ..., -1.1075, -1.0733, -1.0904],
          ...,
          [-0.1486, -0.1486, -0.1314,  ...,  0.9132,  0.9303,  0.9303],
          [-0.0116, -0.0287,  0.0227,  ...,  0.9132,  0.9132,  0.9132],
          [ 0.1768,  0.1426,  0.1939,  ...,  0.9303,  0.9303,  0.8961]],
 
         [[-0.7227, -0.7577, -0.7577,  ..., -0.7402, -0.7052, -0.7577],
          [-0.7402, -0.7402, -0.7402,  ..., -0.7227, -0.6877, -0.7227],
          [-0.7227, -0.7227, -0.7402,  ..., -0.7227, -0.6877, -0.7227],
          ...,
          [-0.0049, -0.0049,  0.0126,  ...,  1.2031,  1.2206,  1.2206],
          [ 0.1527,  0.1176,  0.1702,  ...,  1.2206,  1.2206,  1.2206],
          [ 0.3277,  0.2927,  0.3452,  ...,  1.2381,  1.2381,  1.2556]],


In [21]:
model = Swinv2ForImageClassification.from_pretrained(
    model_name, 
    label2id=label2id,
    id2label=id2label,
    ignore_mismatched_sizes = True, # provide this in case you're planning to fine-tune an already fine-tuned checkpoint
)

Some weights of Swinv2ForImageClassification were not initialized from the model checkpoint at microsoft/swinv2-tiny-patch4-window8-256 and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([1000, 768]) in the checkpoint and torch.Size([4, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([4]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [48]:
from transformers import TrainingArguments, Trainer

batch_size = 8

model_checkpoint = model_name.split("/")[-1]

args = TrainingArguments(
    f"{model_checkpoint}-finetuned-eurosat",
    remove_unused_columns=False,
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=20,
    warmup_ratio=0.1,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    push_to_hub=True,
)

In [49]:
import numpy as np

# the compute_metrics function takes a Named Tuple as input:
# predictions, which are the logits of the model as Numpy arrays,
# and label_ids, which are the ground-truth labels as Numpy arrays.
def compute_metrics(eval_pred):
    """Computes accuracy on a batch of predictions"""
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return metric.compute(predictions=predictions, references=eval_pred.label_ids)

In [50]:
import torch

def collate_fn(examples):
    pixel_values = torch.stack([example["pixel_values"] for example in examples])
    labels = torch.tensor([example["label"] for example in examples])
    return {"pixel_values": pixel_values, "labels": labels}

In [ ]:
from huggingface_hub import login

login()

In [51]:
from datasets import load_metric

metric = load_metric("accuracy")

c:\Users\User\anaconda3\envs\my_env_py311\Lib\site-packages\datasets\load.py:759: FutureWarning: The repository for accuracy contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.1/metrics/accuracy/accuracy.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [52]:
trainer = Trainer(
    model,
    args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=image_processor,
    compute_metrics=compute_metrics,
    data_collator=collate_fn,
)

In [53]:
train_results = trainer.train()
# rest is optional but nice to have
trainer.save_model()
trainer.log_metrics("train", train_results.metrics)
trainer.save_metrics("train", train_results.metrics)
trainer.save_state()

  0%|          | 10/3340 [00:04<21:53,  2.54it/s]
                                                 

  0%|          | 10/3340 [00:04<21:53,  2.54it/s]


{'loss': 0.4103, 'grad_norm': 10.960575103759766, 'learning_rate': 1.4970059880239521e-06, 'epoch': 0.06}


  1%|          | 20/3340 [00:08<21:42,  2.55it/s]
                                                 

  1%|          | 20/3340 [00:08<21:42,  2.55it/s]


{'loss': 0.3885, 'grad_norm': 7.192625522613525, 'learning_rate': 2.9940119760479042e-06, 'epoch': 0.12}


  1%|          | 30/3340 [00:12<21:44,  2.54it/s]
                                                 

  1%|          | 30/3340 [00:12<21:44,  2.54it/s]


...

{'loss': 0.26, 'grad_norm': 7.062151908874512, 'learning_rate': 0.0, 'epoch': 19.91}





































































































































                     


                                                   

                                           

100%|██████████| 3340/3340 [30:39<00:00,  1.98it/s]




{'eval_loss': 0.6760039925575256, 'eval_accuracy': 0.8169685414680649, 'eval_runtime': 7.2762, 'eval_samples_per_second': 144.168, 'eval_steps_per_second': 18.141, 'epoch': 19.91}



                                                   

100%|██████████| 3340/3340 [30:40<00:00,  1.81it/s]


{'train_runtime': 1840.4929, 'train_samples_per_second': 58.289, 'train_steps_per_second': 1.815, 'train_loss': 0.34975095369858655, 'epoch': 19.91}
***** train metrics *****
  epoch                    =      19.9106
  total_flos               = 3236336816GF
  train_loss               =       0.3498
  train_runtime            =   0:30:40.49
  train_samples_per_second =       58.289
  train_steps_per_second   =        1.815


In [54]:
metrics = trainer.evaluate()
# some nice to haves:
trainer.log_metrics("eval", metrics)
trainer.save_metrics("eval", metrics)

100%|██████████| 132/132 [00:06<00:00, 19.21it/s]

***** eval metrics *****
  epoch                   =    19.9106
  eval_accuracy           =     0.8227
  eval_loss               =     0.6331
  eval_runtime            = 0:00:07.14
  eval_samples_per_second =    146.869
  eval_steps_per_second   =     18.481


In [198]:
# rt load_dataset 
# dataset = load_dataset("imagefolder", data_files=["data/train/anger", "data/train/happy", "data/train/panic", "data/train/sadness" ])
# # dataset = load_dfrom datasets impoataset("imagefolder", data_dir="data/train/anger")

FileNotFoundError: Unable to find 'C:/Users/User/Desktop/Code/Github/Final_project\data/train/happy'

In [182]:
# dataset['train'].features += 0
# dataset['train'].features['label'] = 0
# dataset['train']

Dataset({
    features: ['image'],
    num_rows: 1373
})

In [ ]:
# df = pd.DataFrame(json_train, columns=['filename', 'faceExp_uploader'])
# df['faceExp_uploader'].unique()
# train_npz = np.load(TRAIN_SEGMENT_PATH + 'train_anger.npz')
# file_name = json_train[0]['filename']
# plt.imshow(train_npz[file_name], vmin=0, vmax=5)